In [65]:
import pandas as pd
import numpy as np
from pprint import pprint

In [40]:
column_names =['user_id','item_id','rating','timestamp']
df = pd.read_csv('./ml-100k/u.data', sep='\t',names=column_names)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [37]:
df2 = pd.read_csv('./ml-100k/u.item', sep='\|',encoding = "ISO-8859-1")
df2.head()

<ipython-input-37-ec7f4e87e5e3>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df2 = pd.read_csv('./ml-100k/u.item', sep='\|',encoding = "ISO-8859-1")


,1,Toy Story (1995),01-Jan-1995,Unnamed: 3,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,0.1,0.2,1.1,1.2,...,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Content Filtering Recommendation Process

In [39]:
#loadin the movie and rating 
def loadMovieLens(path='./ml-100k'):
    # Get movie titles
    movies={}
    for line in open(path+'/u.item',encoding = "ISO-8859-1"):
        (id,title)=line.split('|')[0:2]
        movies[id]=title
    # Load data
    prefs={}
    for line in open(path+'/u.data'):
        (user,movieid,rating,ts)=line.split('\t')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
    return prefs

In [55]:
#This function calculates Euclidean Distance Score
def sim_distance(prefs,person1,person2):
    # Get the list of shared_items
    si={}
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item]=1
    # if they have no ratings in common, return 0
    if len(si)==0: return 0
    # Add up the squares of all the differences

    sum_of_squares=sum([pow(prefs[person1][item]-prefs[person2][item],2) for item in prefs[person1] if item in prefs[person2]])
    # print(sum_of_squares)
    return 1/(1+sum_of_squares)


In [56]:
#Returns top score
def topMatches(prefs,person,n=5,similarity=sim_distance):
    scores=[(similarity(prefs,person,other),other) for other in prefs if other!=person]
    # print(scores)
    # Sort the list so the highest scores appear at the top
    scores.sort()
    scores.reverse()
    return scores[0:n]

In [57]:
#This transform the dataset{user1:[{movie1:rating,...},movie2:....]} into {movie1:[{user1_rating,..},movie2:...]}
def transformPrefs(prefs):
    result={}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
            # Flip item and person
            result[item][person]=prefs[person][item]
    return result

In [58]:
# its calculated how similar is the movie to on another based on rating
def calculateSimilarItems(prefs,n=10):
    # Create a dictionary of items showing which other items they
    # are most similar to.
    result={}
    # Invert the preference matrix to be item-centric
    # print(prefs)
    # print('----------------------------------')
    itemPrefs=transformPrefs(prefs)
    # print(itemPrefs)
    c=0
    for item in itemPrefs:
        # Status updates for large datasets
        c+=1
        if c%100==0: pass
        # Find the most similar items to this one
        scores=topMatches(itemPrefs,item,n=n,similarity=sim_distance)
        result[item]=scores
    return result

In [59]:
#Gives recommendation on Content Basis
def getContentBasedRecommended(prefs,itemMatch,user):
    userRatings=prefs[user]
    scores={}
    totalSim={}
    # Loop over items rated by this user
    for (item,rating) in userRatings.items( ):
        # Loop over items similar to this one
        for (similarity,item2) in itemMatch[item]:
            # Ignore if this user has already rated this item
            if item2 in userRatings: continue
            # Weighted sum of rating times similarity
            scores.setdefault(item2,0)
            scores[item2]+=similarity*rating
            # Sum of all the similarities
            totalSim.setdefault(item2,0)
            totalSim[item2]+=similarity
    # Divide each total score by total weighting to get an average
    rankings=[(score/totalSim[item],item) for item,score in scores.items( )]
    # Return the rankings from highest to lowest
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [68]:
#load movie lens dataset
prefs=loadMovieLens()
#prefs['87']

{'Naked Gun 33 1/3: The Final Insult (1994)': 4.0,
 'Con Air (1997)': 4.0,
 'Sabrina (1995)': 4.0,
 'Waterworld (1995)': 4.0,
 'To Wong Foo, Thanks for Everything! Julie Newmar (1995)': 3.0,
 'Clueless (1995)': 4.0,
 'Jurassic Park (1993)': 5.0,
 'Brady Bunch Movie, The (1995)': 2.0,
 'Son in Law (1993)': 4.0,
 'Indiana Jones and the Last Crusade (1989)': 5.0,
 'Good, The Bad and The Ugly, The (1966)': 5.0,
 'Dead Poets Society (1989)': 5.0,
 'Dead Man Walking (1995)': 4.0,
 "Joe's Apartment (1996)": 2.0,
 'GoldenEye (1995)': 4.0,
 'M*A*S*H (1970)': 5.0,
 'Something to Talk About (1995)': 2.0,
 'Lightning Jack (1994)': 3.0,
 'Big Green, The (1995)': 3.0,
 'Cowboy Way, The (1994)': 3.0,
 "Ulee's Gold (1997)": 3.0,
 'Addams Family Values (1993)': 2.0,
 '2001: A Space Odyssey (1968)': 5.0,
 'Platoon (1986)': 3.0,
 'Return of the Pink Panther, The (1974)': 4.0,
 'Four Weddings and a Funeral (1994)': 5.0,
 'Under Siege (1992)': 4.0,
 'Ace Ventura: Pet Detective (1994)': 4.0,
 'Die Hard: Wit

In [61]:
#calculate the similarity between each movies one and another
similarmovies=calculateSimilarItems(prefs)

In [66]:
#get the recommendation for the user Eg getContentBasedRecommended(loaded_dataset,similiarity_dataset,'user_id')
pprint(getContentBasedRecommended(prefs,similarmovies,'87')[0:30])

[(5.0, "What's Eating Gilbert Grape (1993)"),
 (5.0, 'Temptress Moon (Feng Yue) (1996)'),
 (5.0, 'Street Fighter (1994)'),
 (5.0, 'Spice World (1997)'),
 (5.0, 'Sliding Doors (1998)'),
 (5.0, 'Shooting Fish (1997)'),
 (5.0, "Roseanna's Grave (For Roseanna) (1997)"),
 (5.0, 'Rendezvous in Paris (Rendez-vous de Paris, Les) (1995)'),
 (5.0, 'Reluctant Debutante, The (1958)'),
 (5.0, 'Police Story 4: Project S (Chao ji ji hua) (1993)'),
 (5.0, 'Palmetto (1998)'),
 (5.0, 'New York Cop (1996)'),
 (5.0, 'Love Is All There Is (1996)'),
 (5.0, 'Johns (1996)'),
 (5.0, 'Innocents, The (1961)'),
 (5.0, 'Hollow Reed (1996)'),
 (4.666666666666667, 'Stranger, The (1994)'),
 (4.6, 'So Dear to My Heart (1949)'),
 (4.6, 'Saint of Fort Washington, The (1993)'),
 (4.571428571428571, 'They Made Me a Criminal (1939)'),
 (4.533333333333333, 'Silence of the Palace, The (Saimt el Qusur) (1994)'),
 (4.5, 'True Crime (1995)'),
 (4.5, 'Traveller (1997)'),
 (4.5, 'Thousand Acres, A (1997)'),
 (4.5, 'Steel (1997)')